In [1]:
import os

In [2]:
os.getcwd()

'/Users/kyeongchanlee/projects/bigcontest2024/analysis'

In [2]:
os.chdir('../')

In [3]:
os.getcwd()

'/Users/kyeongchanlee/projects/bigcontest2024'

In [4]:
from graphrag.get_embedding_model import get_embedding_model
from graphrag.graph_search import retrieve_top_k_stores_by_review_graph_embedding, process_review_node
from llm_response.langgraph_graph_state import GraphState
import streamlit as st
import re

from prompt.text_to_cypher_for_recomm import EXAMPLES_COMBINED, NEO4J_SCHEMA_RECOMM, TEXT_TO_CYPHER_FOR_RECOMM_TEMPLATE
from utils import get_candidate_str
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

/Users/kyeongchanlee/.pyenv/versions/bigcontest2024/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
from graphrag.retriever import get_neo4j_vector
from config import CONFIG
store_retriever_rev_emb = get_neo4j_vector().as_retriever(search_kwargs={"k": CONFIG.store_retriever_rev_emb_k})

from llm_response.get_llm_model import get_llm_model
from utils import graphdb_driver

# 마크다운에서 HTML로 변환하는 함수
def convert_markdown_to_html(text):
    # **bold** -> <b>bold</b>
    text = re.sub(r'\*\*(.*?)\*\*', r'<b>\1</b>', text)
    return text

def text_to_cypher_for_recomm(llm, state:GraphState):
    print(f"Text2Cypher for RECOMM".ljust(100, '-'))
    response = llm.invoke(
        TEXT_TO_CYPHER_FOR_RECOMM_TEMPLATE.format(
            NEO4J_SCHEMA_RECOMM=NEO4J_SCHEMA_RECOMM,
            EXAMPLES_COMBINED=EXAMPLES_COMBINED, 
            query=state['query']
            )
    )
    cypher = response.content.replace('```', '').replace('cypher', '').strip()
    print(f"cypher : \n{cypher}\n")
    state['t2c_for_recomm'] = cypher
    print(f"# input_tokens count : {response.usage_metadata['input_tokens']}")
    return state

In [7]:
llm = get_llm_model()

In [8]:
state = {}
state['query'] = '바다 보이는 횟집 추천해줘. 제주 신화월드 근처에 부모님 모시고 가기 좋은 집 추천해줘'
# state['intent'] = '60대 부부가 함께 여유롭게 대화를 나누며 편안하게 흑돼지를 즐길 수 있는, 신선한 재료와 아늑하고 조용한 분위기를 갖춘 맛집'

In [9]:
from llm_response.langgraph_nodes.routing.route_and_intent_analysis import route_and_intent_analysis

In [37]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from graphrag.get_embedding_model import get_embedding_model

In [10]:
state = route_and_intent_analysis(llm, state)

2024-11-11 15:25:15.039 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Routing and intent analysis-------------------------------------------------------------------------
query : 바다 보이는 횟집 추천해줘. 제주 신화월드 근처에 부모님 모시고 가기 좋은 집 추천해줘


2024-11-11 15:25:15.274 
  command:

    streamlit run /Users/kyeongchanlee/.pyenv/versions/bigcontest2024/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-11-11 15:25:15.275 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 15:25:15.275 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 15:25:15.275 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 15:25:16.822 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-11 15:25:16.824 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.



```json
{
    'query_type': 'recomm',
    'comment': '😄 이 질문은 추천형 질문으로 분석되었어요.',
    'intent': '부모님과 함께 바다를 보며 즐거운 식사를 할 수 있는, 깔끔하고 쾌적한 분위기의 횟집으로, 신선한 해산물과 다양한 메뉴를 제공하며 가격이 적당한 곳'
}
```
# input_tokens count : 2222


In [11]:
state

{'query': '바다 보이는 횟집 추천해줘. 제주 신화월드 근처에 부모님 모시고 가기 좋은 집 추천해줘',
 'query_type': 'recomm',
 'intent': '부모님과 함께 바다를 보며 즐거운 식사를 할 수 있는, 깔끔하고 쾌적한 분위기의 횟집으로, 신선한 해산물과 다양한 메뉴를 제공하며 가격이 적당한 곳'}

In [12]:
state = text_to_cypher_for_recomm(llm, state)

Text2Cypher for RECOMM------------------------------------------------------------------------------
cypher : 
MATCH (a:ATTR)
WHERE a.AREA_NM CONTAINS "신화" AND a.AREA_NM CONTAINS "월드"
WITH a.latitude AS swLat, a.longitude AS swLon
MATCH (rg:Region)-[:HAS_STORE]->(c:STORE)-[:HAS_VISIT_KEYWORD]->(v:Visit_with)
WHERE c.MCT_TYPE = '일식' AND v.keyword CONTAINS "부모님"
WITH c, swLat, swLon, c.latitude AS storeLat, c.longitude AS storeLon
RETURN c.pk AS pk, c.MCT_NM AS RestaurantName, c.ADDR AS Address, c.menu AS Menu, 
       point.distance(point({latitude: swLat, longitude: swLon}), point({latitude: storeLat, longitude: storeLon})) AS Distance_in_meters_from_Jeju_Shinhwa_World
ORDER BY Distance_in_meters_from_Jeju_Shinhwa_World ASC
LIMIT 500

# input_tokens count : 3535


In [13]:
state

{'query': '바다 보이는 횟집 추천해줘. 제주 신화월드 근처에 부모님 모시고 가기 좋은 집 추천해줘',
 'query_type': 'recomm',
 'intent': '부모님과 함께 바다를 보며 즐거운 식사를 할 수 있는, 깔끔하고 쾌적한 분위기의 횟집으로, 신선한 해산물과 다양한 메뉴를 제공하며 가격이 적당한 곳',
 't2c_for_recomm': 'MATCH (a:ATTR)\nWHERE a.AREA_NM CONTAINS "신화" AND a.AREA_NM CONTAINS "월드"\nWITH a.latitude AS swLat, a.longitude AS swLon\nMATCH (rg:Region)-[:HAS_STORE]->(c:STORE)-[:HAS_VISIT_KEYWORD]->(v:Visit_with)\nWHERE c.MCT_TYPE = \'일식\' AND v.keyword CONTAINS "부모님"\nWITH c, swLat, swLon, c.latitude AS storeLat, c.longitude AS storeLon\nRETURN c.pk AS pk, c.MCT_NM AS RestaurantName, c.ADDR AS Address, c.menu AS Menu, \n       point.distance(point({latitude: swLat, longitude: swLon}), point({latitude: storeLat, longitude: storeLon})) AS Distance_in_meters_from_Jeju_Shinhwa_World\nORDER BY Distance_in_meters_from_Jeju_Shinhwa_World ASC\nLIMIT 500'}

In [14]:
records, summary, keys = graphdb_driver.execute_query(state['t2c_for_recomm'])

In [15]:
print(state['t2c_for_recomm'])

MATCH (a:ATTR)
WHERE a.AREA_NM CONTAINS "신화" AND a.AREA_NM CONTAINS "월드"
WITH a.latitude AS swLat, a.longitude AS swLon
MATCH (rg:Region)-[:HAS_STORE]->(c:STORE)-[:HAS_VISIT_KEYWORD]->(v:Visit_with)
WHERE c.MCT_TYPE = '일식' AND v.keyword CONTAINS "부모님"
WITH c, swLat, swLon, c.latitude AS storeLat, c.longitude AS storeLon
RETURN c.pk AS pk, c.MCT_NM AS RestaurantName, c.ADDR AS Address, c.menu AS Menu, 
       point.distance(point({latitude: swLat, longitude: swLon}), point({latitude: storeLat, longitude: storeLon})) AS Distance_in_meters_from_Jeju_Shinhwa_World
ORDER BY Distance_in_meters_from_Jeju_Shinhwa_World ASC
LIMIT 500


In [17]:
len(records)

73

In [159]:
def retrieve_top_similar_stores_pk(store_pk, query_embedding):
    """
    특정 STORE 노드에 연결된 리뷰 중 각 STORE별 유사도가 가장 높은 리뷰를 반환합니다.
    """
    query = """
    MATCH (s:STORE)-[:HAS_REVIEW]->(r:Review)
    WHERE r.textEmbedding IS NOT NULL
        AND s.pk IN $store_pk
    WITH s.pk AS pk, r.text AS text, 
         gds.similarity.cosine(r.textEmbedding, $query_embedding) AS similarity
    ORDER BY similarity DESC
    WITH pk, collect({text: text, similarity: similarity})[0] AS top_review
    RETURN pk,
           top_review.similarity AS similarity
    LIMIT 6
    """
    with driver.session() as session:
        start_time = time.time()
        result = session.run(query, store_pk=store_pk, query_embedding=query_embedding)
        end_time = time.time()
        print(f"Query Execution Time for stores {len(store_pk)}: {end_time - start_time:.4f} seconds")
        return [
            {
                "pk": record["pk"],
                "similarity": record["similarity"]
            } for record in result
        ]


In [160]:
top_sim_stores = retrieve_top_similar_stores_pk(store_pk=[r['pk'] for r in records], query_embedding=query_embedding)

Query Execution Time for stores 73: 0.0438 seconds


In [169]:
top_sim_stores

[{'pk': 6380, 'similarity': 0.8109827854829943},
 {'pk': 6263, 'similarity': 0.7879735429710301},
 {'pk': 8156, 'similarity': 0.7820395688080898},
 {'pk': 5031, 'similarity': 0.7751468554055075},
 {'pk': 3990, 'similarity': 0.7677706777407783},
 {'pk': 785, 'similarity': 0.767398829003102}]

In [161]:
top_sim_pks = [ts['pk'] for ts in top_sim_stores]

In [162]:
top_sim_pks

[6380, 6263, 8156, 5031, 3990, 785]

In [166]:
records[0]['pk']

4971

In [170]:
candidates_2nd= [r for r in records if r['pk'] in top_sim_pks]

In [172]:
def retrieve_top_k_reviews(store_pk, query_embedding, driver, k=3):
    """
    특정 STORE 노드에 연결된 리뷰 중 유사한 TOP-K 리뷰를 반환합니다.
    """
    query = """
    MATCH (s:STORE {pk: $store_pk})-[:HAS_REVIEW]->(r:Review)
    WHERE r.textEmbedding IS NOT NULL
    RETURN r.text AS text, gds.similarity.cosine(r.textEmbedding, $query_embedding) AS similarity
    ORDER BY similarity DESC
    LIMIT $k
    """
    with driver.session() as session:
        result = session.run(
            query, store_pk=store_pk, query_embedding=query_embedding, k=k
        )
        return [
            {"text": record["text"], "similarity": record["similarity"]}
            for record in result
        ]

In [188]:
cypher_result_str = ''
for r in candidates_2nd:
    r_keys = r.keys()
    one_record_str = ''
    for key in r_keys:
        one_record_str += f"{key} : {str(r[key])[:100]}\n"
        if key == 'pk':
            reviews = retrieve_top_k_reviews(r[key], query_embedding, graphdb_driver, k=2)
            if reviews:
                reviews_lst = [f"{ri}. {review['text'][:100]}".strip() for ri, review in enumerate(reviews, start=1)]
                one_record_str += "리뷰 : \n" + '\n'.join(reviews_lst) + "\n"
    if '리뷰' in one_record_str:
        t2c_candidates_cnt += 1
        cypher_result_str += one_record_str + '\n'

In [189]:
print(cypher_result_str)

pk : 3990
리뷰 : 
1. 부모님 모시고 가기 딱 좋은곳!!
유튜브로 처음 알게되어 가야지 가야지하다가 예약하고갔습니다.
테이블 공간이 넓어서 소란스럽다고 생각이안들고, 특히 음식이 맛있었습니다. 회가 쫄깃
2. 너무 깔끔하고 정갈하게 나와서 맛있게 먹었어요. 횟집 가면 여러명이서 다양한 메뉴 시켜먹기에 부담이 있는데 (특히 해물류) 여기는 코스식으로 이것저것 맛볼 수 있어서 진짜 좋았어요
RestaurantName : 색달바당
Address : 제주 서귀포시 색달동 1898-1번지 1층
Menu : 바당 2인 코스:120000, 바당 3인 코스:150000, 바당 4인 코스:200000, 바당 5인 코스:250000, 바당 6인 코스:300000, 제주 돌돔:300000, 제
Distance_in_meters_from_Jeju_Shinhwa_World : 10179.789540271771

pk : 8156
리뷰 : 
1. 물놀이 후, 횟집왔는데 너무 맛있게 잘 먹었어요! 좋은사람들과 좋은 회 먹으면서 좋은시간 보냈어요^^
2. 부모님 모시고 대포항 갔다가 따로 찾아보진 않고 회정식 먹으러 갔어요. 바다 바로 앞이라 뷰도 좋고 깔끔했어요. 가격대는 조금 있지만 풍경값과 회정식값 했네요 잘 먹었습니다^^
RestaurantName : 큰갯물횟집
Address : 제주 서귀포시 대포동 2070번지
Menu : 🐟자연산활어류:0, 🐠통갈치구이:80000, 🐟한치회,고등어회,갈치회 각:50000, 🦞매운탕:20000, 갈치조림:60000, 제철회 히라스 , 방어회:0, 갓돔:0, 다금바리:
Distance_in_meters_from_Jeju_Shinhwa_World : 13423.130278202529

pk : 5031
리뷰 : 
1. 부모님 모시고 갔는데, 매우 맛있게 드셨어요. 연어집에서 파스타와 안심카츠는 처음 먹어봤는데~ 우와~~~ 진짜 넘넘 맛있네요. 부모님께서 연돈 돈까스 드시고 오셨었는데, 이거 드시
2. 연어집 답게 연어가 정말 맛있어요!!!